## (B)  Build a minimal generative model and use it to analyze test data

### 1. Import your package in the documentation notebook.

In [3]:
import lennardjones

### 2. Describe your generative model.

#### We will start with just the Lennard Jones model.

The Lennard Jones *potential* has a well known phenominolgical model:

$$ U_\text{LJ} = 4 \epsilon \left ( \frac{\sigma^{12}}{z^{12}} - \frac{\sigma^6}{z^6} \right ) $$

Where $\epsilon$ is the depth of the well (or 'strength' of the interaction), $\sigma$ is distance when the potential is 0 (or the 'size' of the interacting particles), and $z$ is the distance between the particles.

To compute the force, $F = -\partial U / \partial z$, so 
$$ F_\text{LJ} = 4 \epsilon \left ( \frac{12\sigma^{12}}{z^{13}} - \frac{6\sigma^6}{z^7} \right ) $$

The goal of the first task is to fit to the above model, using $\epsilon$ and $\sigma$ as parameters. 

However, the measured data is frequency shift (df) and must be first coverted to force. This is done using the Sader-Jarvis method, which takes an intergral over the distance data, starting at the closest $z$ position. 

All parameters in the Sader-Jarvis formula are known experimental parameters. 

#### Sader-Jarvis method to convert to force

$$ F_\text{meas} = 2k \int_z^\inf \left ( 1 +  \frac{a^{1/2}}{8 \sqrt{\pi(t-z)}} \right ) d\Omega(t) - \frac{a^{3/2}}{\sqrt{2(t-z)}} \frac{d \Omega(t)}{dt} dt $$

where $k$ is the spring constant of the tuning fork, $a$ is the oscillation amplitude, $\Omega(z) = d\omega(z)/ \omega_0 = d\text{f}(z)/\text{f}_0$, and $t$ is the integration variable.

$k$ and $a$ are experiemental details that are known.


#### Experimental considerations

The raw data is frequency shift df and position of the piezo $z'$. 

**df**

The frequency shift is the measured data and has noise. Based on traces taken far from the sample, the background noise on this measurement is $\sigma_\text{df} = 0.5$ Hz. 

***z'***

To keep things simple the first time, we will set the position of the sample $z_0$ equal to 0.5 nm below the lowest $z'$ position. Thus the tip position relative to the sample (i.e. the $z$ in the above equations) becomes $z$ = $z'$ -   $z_\text{min}$ - 0.5 nm.

$k$ = 1800 N/m

$a$ = 0.5 nm

In all honesty it is quite possible there is a 10 % uncertainty on each of these parameters as well. It may be desirable to include that as a prior and marginalize out the small range.

However, again to keep things simple in this first itteration, we will assume the values above are exact. 

Given all of this, there are *no* parameters in the force calculation and so we should simple be able to perform the calculation; however, the uncertainty needs to also be converted from df to force (i.e. $\sigma_\text{df} \rightarrow \sigma_\text{force}$ )

##### Use the notation we introduced in the course to describe your statistical model, which should include the priors

I'm a little stuck on this because of the Sader-Jarvis conversion on the y-data. After that however, it would be:

$\epsilon$ ~ Uniform(0,$\inf$)

$\sigma$ ~ Uniform(0,$\inf$)

$\mu$ = 4 $\epsilon$ (12$\sigma^{12}$/$z^{13}$ - 6$\sigma^{6}$/$z^{7}$)

F ~ normal($\mu$ = $\mu$, $\sigma = \sigma_\text{force}$ )